In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# trackML challenge 

This notebook used to generate inputs and train the model, both model and the input reprocessing are defined in `sample_code_submission/`

#### optimization:
to optimize the code try `!cython preprocess.pyx -a`

In this notebook we will examine when we can merge close hits, and we will validate the evaluation of the cython code for data preparation:


**[Hit merging](#merging)** - evaluation of criteria for close hits that share the same track

**[Validation](#validation)** - validation of the cython code




In [60]:
from trackml.dataset import load_dataset
PATH_TO_DATA = "/home/data/train_sample_single"

In [61]:
data = load_dataset(PATH_TO_DATA, parts=['hits', 'cells', 'truth', 'particles'])
event_id, hits, cells, truth, particles = next(data)

Index(['hit_id', 'particle_id', 'tx', 'ty', 'tz', 'tpx', 'tpy', 'tpz',
       'weight'],
      dtype='object')

In [70]:
old_keys = truth.keys()
new_truth = truth.merge(hits[['hit_id','volume_id']])
mask = ((new_truth['volume_id']==8) | (new_truth['volume_id']==13) | (new_truth['volume_id']==18))
new_truth = new_truth.loc[mask][old_keys]

<a id='merging'></a>

## Hit merging:


Check for the criteria to merge hits (some of the hits in the layer 1 will originating from the same particle, therefore we will assign same label (`track_id`) to those hits

Lets obtain the distribtion to decide on the cut criteria

In [4]:
def getLayer(r, z, volume_id, layer_id):   
    if volume_id==8 and layer_id==2:
        return 1
    elif volume_id==9:
        return 2
    elif volume_id==7:
        return 2
    else: return -1

In [ ]:
hits['r'] = np.sqrt(hits['x']**2 + hits['y']**2)
hits['phi'] = np.arctan2(hits['y'],hits['x'])
hits['theta'] = np.arctan2(hits['r'],hits['z'])
hits['eta'] = -np.log(np.tan(0.5*hits['theta']))
hits['evtid'] = event_id
hits['layer'] = hits.apply(lambda x: getLayer(x['r'],x['z'],x['volume_id'],x['layer_id']), axis=1)
keys = ['hit_id','z','phi','r','layer']
hits = (hits[keys].merge(truth[['hit_id','particle_id']], on='hit_id'))

Sudy the hits from the first layer only (separate barrel with end-caps):

Let's order the hits in increasing order in $\phi$, and look on a neighbour hits.
By construction, these two hits are close in $\phi$, the question which criteria one should apply to verify that these two hits are comming form the same track:

In [ ]:
filter_hits = hits.loc[hits['layer']==1]

In [ ]:
filtered_sorted_hits = filter_hits.sort_values(by=['phi'])
n = filtered_sorted_hits.shape[0]
dphi = []
dz = []
dr = []
is_true = []
for i in range(n-1):
    is_true.append( filtered_sorted_hits.particle_id.values[i] == filtered_sorted_hits.particle_id.values[i+1]  )
    dphi.append( filtered_sorted_hits.phi.values[i+1] - filtered_sorted_hits.phi.values[i]  ) 
    dz.append( filtered_sorted_hits.z.values[i+1] - filtered_sorted_hits.z.values[i]  ) 
    dr.append( filtered_sorted_hits.r.values[i+1] - filtered_sorted_hits.r.values[i]  ) 
dphi = np.array(dphi)
dz = np.array(dz)
dr = np.array(dr)
is_true = np.array(is_true)

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 3, 1)
ax.hist(dphi[(is_true==False) ],100,(-0.001,0.006),label='false')
ax.hist(dphi[(is_true==True) ],100,(-0.001,0.006),label='true',log=True)
ax.set_xlabel(r'$\Delta\phi [rad]$',fontsize=20)

ax = fig.add_subplot(1, 3, 2)
ax.hist(dr[(is_true==False) & (dphi<0.002) ],100,(-1,1),label='false')
ax.hist(dr[(is_true==True) & (dphi<0.002) ],100,(-1,1),label='true', log=True)
ax.set_xlabel(r'$\Delta R [mm]$',fontsize=20)

ax = fig.add_subplot(1, 3, 3)
ax.hist(dz[(is_true==False) & (np.fabs(dr)>0.025) & (dphi<0.002)],100,(-20,20),label='false')
ax.hist(dz[(is_true==True) & (np.fabs(dr)>0.025) & (dphi<0.002)],100,(-20,20),label='true',alpha=0.4)
ax.set_xlabel(r'$\Delta z [mm]$',fontsize=20)

plt.legend()
plt.savefig('Merge_hits_Barrel.pdf')
plt.show()   


In [ ]:
filter_hits = hits.loc[hits['layer']==2]

In [ ]:
filtered_sorted_hits = filter_hits.sort_values(by=['phi'])
n = filtered_sorted_hits.shape[0]
dphi = []
dz = []
dr = []
is_true = []
for i in range(n-1):
    is_true.append( filtered_sorted_hits.particle_id.values[i] == filtered_sorted_hits.particle_id.values[i+1]  )
    dphi.append( filtered_sorted_hits.phi.values[i+1] - filtered_sorted_hits.phi.values[i]  ) 
    dz.append( filtered_sorted_hits.z.values[i+1] - filtered_sorted_hits.z.values[i]  ) 
    dr.append( filtered_sorted_hits.r.values[i+1] - filtered_sorted_hits.r.values[i]  ) 
dphi = np.array(dphi)
dz = np.array(dz)
dr = np.array(dr)
is_true = np.array(is_true)

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 3, 1)
ax.hist(dphi[(is_true==False) ],100,(-0.001,0.006),label='false')
ax.hist(dphi[(is_true==True) ],100,(-0.001,0.006),label='true',log=True)
ax.set_xlabel(r'$\Delta\phi [rad]$',fontsize=20)

ax = fig.add_subplot(1, 3, 2)
ax.hist(dz[(is_true==False) & (dphi<0.001)],100,(-1,1),label='false')
ax.hist(dz[(is_true==True) & (dphi<0.001)],100,(-1,1),label='true', log=True)
ax.set_xlabel(r'$\Delta z [mm]$',fontsize=20)

ax = fig.add_subplot(1, 3, 3)
ax.hist(dr[(is_true==False) & (dphi<0.001) & (np.fabs(dz)>0.25) ],100,(-5,5),label='false')
ax.hist(dr[(is_true==True) & (dphi<0.001) & (np.fabs(dz)>0.25) ],100,(-5,5),label='true',alpha=0.6, log=True)
ax.set_xlabel(r'$\Delta R [mm]$',fontsize=20)


plt.legend()
plt.savefig('Merge_hits_EC.pdf')
plt.show()   


<a id='validation'></a>

## Validation of segment selection:

consider a single (eta,phi) window, extract tracks using the code, and usigg the dataframes:

In [5]:
phi_bins = np.linspace(0,0.5,2)
eta_bins = np.linspace(0,0.1,2)
data = load_dataset(PATH_TO_DATA, parts=['hits', 'cells', 'truth', 'particles'])
event_id, hits, cells, truth, particles = next(data)

Add layer index to the hits dataframe

In [6]:
def getLayer(volume_id, layer_id):   
    if volume_id==8:
        return layer_id/2
    elif volume_id==13:
        return layer_id/2 + 4
    elif volume_id==17:
        return layer_id/2 + 8
    elif volume_id==7:
        return 8 - layer_id/2
    elif volume_id==9:
        return layer_id/2
    elif volume_id==12:
        return 7 - layer_id/2
    elif volume_id==14:
        return layer_id/2
    elif volume_id==16:
        return 7 - layer_id/2
    elif volume_id==18:
        return layer_id/2
    else: return -1

In [7]:
hits['layer'] = hits.apply(lambda x: getLayer(x['volume_id'],x['layer_id']), axis=1)

Get the truth info from the `cells` and `truth` dataframes:

In [8]:
merged_hits = hits.merge(truth[['hit_id','particle_id']], on='hit_id')[['hit_id','x','y','z','particle_id','layer']].copy()

In [10]:
merged_hits['evtid'] = event_id
merged_hits['r'] = np.sqrt(merged_hits['x']**2 + merged_hits['y']**2)
theta = np.arctan2(merged_hits['r'],merged_hits['z'].values)
merged_hits['phi'] = np.arctan2(merged_hits['y'],merged_hits['x'])
merged_hits['eta'] = -np.log(np.tan(0.5*theta))

In [11]:
mask = (merged_hits['eta']>eta_bins[0]) & (merged_hits['eta']<eta_bins[1]) \
     & (merged_hits['phi']>phi_bins[0]) & (merged_hits['phi']<phi_bins[1])
filtered_hits = merged_hits.loc[mask]
print('remanining',filtered_hits.shape[0],'hits')

remanining 119 hits


In [12]:
filtered_hits.head(4)

,hit_id,x,y,z,particle_id,layer,evtid,r,phi,eta
18092,18093,29.371799,12.75130,0.903263,337777806073135104,1.0,21100,32.020279,0.409582,0.028205
18096,18097,30.642099,9.11798,0.409374,166643425414742016,1.0,21100,31.969921,0.289220,0.012805
19607,19608,28.916300,12.55390,0.673269,337777806073135104,1.0,21100,31.523846,0.409592,0.021356
19618,19619,30.168600,8.97229,0.445777,166643425414742016,1.0,21100,31.474537,0.289074,0.014163


Next lines will calculate the sigments using pandas dataframes

In [13]:
z0_max = 150
pt_min =  180
rho_min = pt_min/0.6


def calc_dphi(hit_pairs):
    dphi = hit_pairs.phi_1 - hit_pairs.phi_2
    dphi = np.arccos(np.cos(dphi))
    return dphi

def calc_dr(hit_pairs):
    dx = hit_pairs.x_2 - hit_pairs.x_1
    dy = hit_pairs.y_2 - hit_pairs.y_1
    return np.sqrt(dx**2 + dy**2)   

def select_segments(hits1, hits2, rho_min, z0_max):
    # Start with all possible pairs of hits
    keys = ['evtid', 'r', 'x', 'y', 'phi', 'z']
    hit_pairs = hits1[keys].reset_index().merge(
        hits2[keys].reset_index(), on='evtid', suffixes=('_1', '_2'))
    
    # Compute circle radius of two points that cross the origin:
    dphi = calc_dphi(hit_pairs)
    dr = calc_dr(hit_pairs)
    rho = 0.5 * dr / dphi
    
    # compute line through the points in z direction:
    dz = hit_pairs.z_2 - hit_pairs.z_1
    tanL = -dz / dr
    z0_1 = hit_pairs.z_1 + hit_pairs.r_1 * tanL
    z0_2 = hit_pairs.z_2 + hit_pairs.r_2 * tanL
    #    0.5(hit_pairs.z_2 + hit_pairs.z_1) + 0.5*(hit_pairs.r_2 + hit_pairs.r_1)*tanL
    
    # Filter segments according to criteria
    good_seg_mask = (rho.abs() > rho_min) & (z0_1.abs() < z0_max)  & (z0_2.abs() < z0_max)
    return hit_pairs[['index_1', 'index_2']][good_seg_mask]

In [14]:
n_det_layers = 10
hits_for_graphs = filtered_hits.loc[filtered_hits['layer']<n_det_layers+1]

In [15]:
l = np.arange(n_det_layers)
layer_pairs = np.stack([l[:-1]+1, l[1:]+1], axis=1)
layer_groups = hits_for_graphs.groupby('layer')
segments = []
for (layer1, layer2) in layer_pairs:
    print('evaluate for',layer1,'-',layer2,end=", ",flush=False)
    hits1 = layer_groups.get_group(layer1)
    hits2 = layer_groups.get_group(layer2)
    segments.append(select_segments(hits1, hits2, rho_min, z0_max))
segments = pd.concat(segments)
print('Done')

evaluate for 1 - 2, evaluate for 2 - 3, evaluate for 3 - 4, evaluate for 4 - 5, evaluate for 5 - 6, evaluate for 6 - 7, evaluate for 7 - 8, evaluate for 8 - 9, evaluate for 9 - 10, Done


In [16]:
n_hits = hits_for_graphs.shape[0]
n_edges = segments.shape[0]
print('n_hits =',n_hits,'n_edges =',n_edges)

n_hits = 119 n_edges = 968


Prepare the graph matrices

In [53]:
feature_names = ['x', 'y', 'z','phi','eta']
X = (hits_for_graphs[feature_names].values).astype(np.float32)
hit_id = (hits_for_graphs['hit_id'].values).astype(np.int32)
pid = (hits_for_graphs['particle_id'].values).astype(np.int64)
y = np.zeros(n_edges, dtype=np.float32)
Is = np.zeros((n_edges, 2), dtype=np.int16)

Use a series to map hit label-index onto positional-index and use use `particle_id` to label connected nodes

In [18]:
hit_idx = pd.Series(np.arange(n_hits), index=hits_for_graphs.index)
seg_start = hit_idx.loc[segments.index_1].values[:,None]
seg_end = hit_idx.loc[segments.index_2].values[:,None]

In [19]:
pid1 = hits_for_graphs.particle_id.loc[segments.index_1].values
pid2 = hits_for_graphs.particle_id.loc[segments.index_2].values
Is = np.concatenate([seg_start,seg_end],axis=1)
y[:] = (pid1 == pid2)
y[pid1 ==0 ] = 0
print('Y stats: connected - ',(y==1).sum(),' disconnected - ',(y==0).sum(),' total -',y.size)

Y stats: connected -  29  disconnected -  939  total - 968


In [20]:
#using the code:
from preprocess import preprocess
list_X, list_Is, list_hits_id, list_labels = preprocess(hits.copy(),phi_bins, eta_bins)
list_X, list_Is, list_hits_id, list_labels = list_X[0], list_Is[0], list_hits_id[0], list_labels[0]
print('n nodes = ',np.concatenate(list_X).shape[0],' n edges = ',np.concatenate(list_Is).shape[0])

n nodes =  119  n edges =  899


Make comparison between selected connections between the nodes, using both methods:

From method #1 we have $n_\text{edges}\times 2$ matrix $I_s$ contains the position of node from matrix $X$

From method #2 we have $9$ matrices $I_s$ and $10$ matrices $X$ correspond to the nodes form each layer and the links betwen them. The links are given as $pos1, pos2$ where the possition are correspond to the rows of matrices $X[0]$ and $X[1]$

let's compare the first 3 nodes connections, for first 3 nodes we have 11 links:

method #1:

In [24]:
print('features of node 1,2,3 and node N connected to node 1,2,3:')
print(' node 0 is ', '%2.2f %2.2f %2.2f %2.2f %2.2f'%tuple(X[0]),' hit_id',hit_id[0],'pid',pid[0])
print(' node 1 is ', '%2.2f %2.2f %2.2f %2.2f %2.2f'%tuple(X[1]),' hit_id',hit_id[1],'pid',pid[1])
print(' node 2 is ', '%2.2f %2.2f %2.2f %2.2f %2.2f'%tuple(X[2]),' hit_id',hit_id[2],'pid',pid[2])
print(' node 3 is ', '%2.2f %2.2f %2.2f %2.2f %2.2f'%tuple(X[3]),' hit_id',hit_id[3],'pid',pid[3])
print(' node 4 is ', '%2.2f %2.2f %2.2f %2.2f %2.2f'%tuple(X[4]),' hit_id',hit_id[4],'pid',pid[4])

for i in Is[:14]:
    print(i[0],'-> [',i[1],'] = ','%2.2f %2.2f %2.2f %2.2f %2.2f'%tuple(X[i[1]]),' id',hit_id[i[1]],'pid',pid[i[1]])


features of node 1,2,3 and node N connected to node 1,2,3:
 node 0 is  29.37 12.75 0.90 0.41 0.03  hit_id 18093 pid 337777806073135104
 node 1 is  30.64 9.12 0.41 0.29 0.01  hit_id 18097 pid 166643425414742016
 node 2 is  28.92 12.55 0.67 0.41 0.02  hit_id 19608 pid 337777806073135104
 node 3 is  30.17 8.97 0.45 0.29 0.01  hit_id 19619 pid 166643425414742016
 node 4 is  30.56 7.84 2.74 0.25 0.09  hit_id 19636 pid 166638065295556608
0 -> [ 10 ] =  66.08 26.39 3.44 0.38 0.05  id 26210 pid 824163061135835136
0 -> [ 11 ] =  66.05 26.45 0.93 0.38 0.01  id 26219 pid 94586999607918592
0 -> [ 12 ] =  68.03 27.32 2.66 0.38 0.04  id 26229 pid 824163061135835136
0 -> [ 13 ] =  65.29 31.07 4.10 0.44 0.06  id 26233 pid 495400425776742400
0 -> [ 14 ] =  68.13 27.18 2.08 0.38 0.03  id 26237 pid 94586999607918592
1 -> [ 9 ] =  70.78 16.27 1.78 0.23 0.02  id 26209 pid 166638065295556608
2 -> [ 10 ] =  66.08 26.39 3.44 0.38 0.05  id 26210 pid 824163061135835136
2 -> [ 11 ] =  66.05 26.45 0.93 0.38 0.01 

medhod #2

In [25]:
#position of the nodes that were selected previously:
pos = [5, 3, 6, 2, 1]
print('features of node 1,2,3,4,5 and node N connected to node 1,2,3,4,5:')
for i in range(5):
    print(' node %d is '%i, '%2.2f %2.2f %2.2f %2.2f %2.2f'%tuple(list_X[0][pos[i]]),'id',list_hits_id[0][pos[i]])

features of node 1,2,3,4,5 and node N connected to node 1,2,3,4,5:
 node 0 is  29.37 12.75 0.90 0.41 0.03 id 18093
 node 1 is  30.64 9.12 0.41 0.29 0.01 id 18097
 node 2 is  28.92 12.55 0.67 0.41 0.02 id 19608
 node 3 is  30.17 8.97 0.45 0.29 0.01 id 19619
 node 4 is  30.56 7.84 2.74 0.25 0.09 id 19636


In [26]:
for j in range(5):
    for i in list_Is[0].loc[list_Is[0].index_1==pos[j]].index_2.values:
        print(j,'-> [',i,'] = ','%2.2f %2.2f %2.2f %2.2f %2.2f'%tuple(list_X[1][i]),'id',list_hits_id[1][i])


0 -> [ 3 ] =  68.13 27.18 2.08 0.38 0.03 id 26237
0 -> [ 4 ] =  66.08 26.39 3.44 0.38 0.05 id 26210
0 -> [ 5 ] =  66.05 26.45 0.93 0.38 0.01 id 26219
0 -> [ 6 ] =  68.03 27.32 2.66 0.38 0.04 id 26229
0 -> [ 7 ] =  65.29 31.07 4.10 0.44 0.06 id 26233
1 -> [ 2 ] =  70.78 16.27 1.78 0.23 0.02 id 26209
2 -> [ 3 ] =  68.13 27.18 2.08 0.38 0.03 id 26237
2 -> [ 4 ] =  66.08 26.39 3.44 0.38 0.05 id 26210
2 -> [ 5 ] =  66.05 26.45 0.93 0.38 0.01 id 26219
2 -> [ 6 ] =  68.03 27.32 2.66 0.38 0.04 id 26229
2 -> [ 7 ] =  65.29 31.07 4.10 0.44 0.06 id 26233
3 -> [ 2 ] =  70.78 16.27 1.78 0.23 0.02 id 26209
4 -> [ 1 ] =  69.45 15.09 0.15 0.21 0.00 id 26200
4 -> [ 2 ] =  70.78 16.27 1.78 0.23 0.02 id 26209


Check label propogation code:

In [35]:
from GraphBuilder.model_loader import model_loader
from GraphBuilder.model.model import GNNmodel as myModel
from preprocess import weights_to_labels

In [36]:
graph_model = model_loader()
graph_model.set_model(myModel())
graph_model.load_weights('path')

In [38]:
weights = graph_model.fit_predict(list_X, list_Is)
labels = weights_to_labels(list_X, list_Is, weights, list_labels)
labels.shape

(119,)

In [44]:
all_hit_id = np.concatenate(list_hits_id)
for j in range(5):
    print('hit_id',all_hit_id[pos[j]],'label',labels[pos[j]])
    print('connected to')
    for i in list_Is[0].loc[list_Is[0].index_1==pos[j]].index_2.values:
        print(j,'-> [',i,'] = id',list_hits_id[1][i],'labeled with',labels[all_hit_id==list_hits_id[1][i]])


hit_id 18093 label 2
connected to
0 -> [ 3 ] = id 26237 labeled with [2]
0 -> [ 4 ] = id 26210 labeled with [2]
0 -> [ 5 ] = id 26219 labeled with [2]
0 -> [ 6 ] = id 26229 labeled with [2]
0 -> [ 7 ] = id 26233 labeled with [2]
hit_id 18097 label 2
connected to
1 -> [ 2 ] = id 26209 labeled with [2]
hit_id 19608 label 2
connected to
2 -> [ 3 ] = id 26237 labeled with [2]
2 -> [ 4 ] = id 26210 labeled with [2]
2 -> [ 5 ] = id 26219 labeled with [2]
2 -> [ 6 ] = id 26229 labeled with [2]
2 -> [ 7 ] = id 26233 labeled with [2]
hit_id 19619 label 1
connected to
3 -> [ 2 ] = id 26209 labeled with [2]
hit_id 19636 label 1
connected to
4 -> [ 1 ] = id 26200 labeled with [1]
4 -> [ 2 ] = id 26209 labeled with [2]


15